## Note

Last time this was run the model architecture changed to remove the merge and only train on the shape classifier, not the MNIST digits. To go back, uncomment the relavent lines below.

In [1]:
from __future__ import print_function
from scipy.misc import imsave, imresize
import numpy as np
import time
import os
import h5py

from keras.models import Sequential
from keras.layers import *
from keras.layers.advanced_activations import ELU
from keras import backend as K

from keras.models import Model

from keras.layers.convolutional import (
    Convolution2D,
    MaxPooling2D,
    AveragePooling2D
)

from keras.layers.normalization import BatchNormalization

from keras.utils import np_utils

import scipy
from scipy import ndimage
from scipy.ndimage import imread

from keras.datasets import mnist
from keras.models import model_from_json
import keras

from time import sleep

import matplotlib.pyplot as plt
# %matplotlib inline

Using Theano backend.


In [2]:
from helper import make_noise_img, make_box_img, deprocess_image, normalize, preprocess_image

In [3]:
try:
    loop_num = int(sys.argv[1])
    print('loop_num', loop_num)
except ValueError:
    loop_num = 0

In [4]:
# dimensions of the generated pictures 
img_width = 28
img_height = 28

# generated_data = '/home/fredhohman/Desktop/data/binary-net/mnist-box/'
# # generated_data = '/home/fredhohman/Desktop/data/binary-net/mnist-box/quick/'
# # generated_data = '/home/fredhohman/Desktop/data/binary-net/mnist-box/line/'
# generated_data = '/home/fredhohman/Desktop/data/binary-net/mnist-box/shapes/'

# mnist_ex_json_path_1 = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_ex_1.json'
# mnist_ex_h5_path_1 = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_ex_1.h5'
# mnist_ex_json_path_2 = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_ex_2.json'
# mnist_ex_h5_path_2 = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_ex_2.h5'

# mnist_json_api_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_api.json' #not used
# mnist_h5_api_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_api.h5' #not used

# h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_' + str(loop_num-1) + '.h5'
# # h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_quick' + str(loop_num-1) + '.h5'
# # h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_line' + str(loop_num-1) + '.h5'
# h5_path = '/home/fredhohman/Desktop/Github/pnnl-summer-research-notes/binary-net/output/mnist_box_shapes' + str(loop_num-1) + '.h5'

# ### home
# # mnist_ex_json_path = '/Users/fredhohman/Github/pnnl-summer-research-notes/output/mnist_ex.json'
# # mnist_ex_h5_path = '/Users/fredhohman/Github/pnnl-summer-research-notes/output/mnist_ex.h5'

In [5]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [6]:
plt.imshow(X_test[0].squeeze())

In [7]:
#wave 1
from helper import boxify_top_left, boxify_top_right, boxify_bottom_left, boxify_bottom_right
from helper import lineify_top_left, lineify_bottom_right
from helper import circleify_top_left, circleify_bottom_right
from helper import triangulify_top_left, triangulify_bottom_right
#wave 2
from helper import boxify_center, lineify_center, circleify_center, triangulify_center, noiseify
from helper import boxify_center_hollow, lineify_center_horizontal, circleify_center_hollow, triangulify_center_hollow, noiseify_blur
#wave 3
from helper import boxify, lineify, circleify

load training data

In [31]:
# no black box = 0
# black box = 1

x_data = []
y_data = []

### create list of real image paths
num_of_pictures = 10 #5000*2

blank = np.zeros([1,28,28])

for i in range(num_of_pictures):
    ###wave 0
    y_val = 0
    x_data.append(boxify_top_left(np.copy(blank)))
    y_data.append(y_val)
    
#     y_val = y_val + 1
#     x_data.append(boxify_top_right(np.copy(blank)))
#     y_data.append(y_val)
    
#     y_val = y_val + 1
#     x_data.append(boxify_bottom_left(np.copy(blank)))
#     y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(boxify_bottom_right(np.copy(blank)))
    y_data.append(y_val)   
    
    y_val = y_val + 1
    x_data.append(lineify_top_left(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(lineify_bottom_right(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(circleify_top_left(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(circleify_bottom_right(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(triangulify_top_left(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(triangulify_bottom_right(np.copy(blank)))
    y_data.append(y_val)
    
    ###wave 1
    y_val = y_val + 1
    x_data.append(boxify_center(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(lineify_center(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(circleify_center(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(triangulify_center(np.copy(blank)))
    y_data.append(y_val)

    y_val = y_val + 1
    x_data.append(noiseify())
    y_data.append(y_val)
    
    ###wave 2
    y_val = y_val + 1
    x_data.append(boxify_center_hollow(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(lineify_center_horizontal(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(circleify_center_hollow(np.copy(blank)))
    y_data.append(y_val)
    
    y_val = y_val + 1
    x_data.append(triangulify_center_hollow(np.copy(blank)))
    y_data.append(y_val)

    y_val = y_val + 1
    x_data.append(noiseify_blur())
    y_data.append(y_val)
    
    # ###wave 3 random
    # y_val = y_val + 1
    # x_data.append(boxify(np.copy(blank)))
    # y_data.append(y_val)
    
    # y_val = y_val + 1
    # try:
    #     x_data.append(lineify(np.copy(blank)))
    #     y_data.append(y_val)
    # except IndexError:
    #     x_data.append(lineify(np.copy(blank)))
    #     y_data.append(y_val)
    
    # y_val = y_val + 1
    # x_data.append(circleify(np.copy(blank)))
    # y_data.append(y_val)
    
print(y_val+1)

18


In [32]:
# ## box
# num_box = 0
# for j in range(num_of_pictures):
#     try:     
#         x_data.append(boxify_center(np.copy(blank)))
#         y_data.append(0)
#         num_box += 1
#     except:
#         pass
# print('num_box', num_box)

# ### line
# num_line = 0
# if loop_num == 0 or loop_num > 0:
#     for i in range(num_of_pictures):
#         try:
#             x_data.append(lineify_center(np.copy(blank)))
#             y_data.append(1)
#             num_line += 1
#         except:
#             pass
#     print('num_line', num_line)

# ### circle
# num_circle = 0
# if loop_num == 0 or loop_num > 0:
#     for i in range(num_of_pictures):
#         try:
#             x_data.append(circleify_center(np.copy(blank)))
#             y_data.append(2)
#             num_circle += 1
#         except:
#             pass
#     print('num_circle', num_circle)
    
# ### triangular
# num_tri = 0
# if loop_num == 0 or loop_num > 0:
#     for i in range(num_of_pictures):
#         try:
#             x_data.append(triangulify_center(np.copy(blank)))
#             y_data.append(3)
#             num_tri += 1
#         except:
#             pass
#     print('num_tri', num_tri)
    
# ### noise
# num_noise = 0
# if loop_num == 0 or loop_num > 0:
#     for i in range(num_of_pictures):
#         try:
#             x_data.append(noiseify())
#             y_data.append(4)
#             num_noise += 1
#         except:
#             pass
#     print('num_noise', num_noise)

In [33]:
# plt.imshow(lineify(np.copy(X_train[0]))[0])
# plt.show()

In [34]:
nb_classes = y_val+1

X_train_new = np.array(x_data)
y_train_new = np.array(y_data)
Y_train_new = np_utils.to_categorical(y_train_new, nb_classes)
X_train_new.shape

(180, 1, 28, 28)

In [35]:
# plt.imshow(X_train_new[0].squeeze())
# plt.show()

In [36]:
import random
s=list(range(X_train_new.shape[0]))
random.shuffle(s) 
X2 = X_train_new[s]+np.random.random(X_train_new.shape)*0.01
Y2 = Y_train_new[s]

In [37]:
X2.shape
Y2.shape

(180, 18)

**MNIST model has been converted to functional API, need to load MNIST weights here**

In [38]:
batch_size = 8 #32
nb_epoch = 100 #5*2

# input image dimensions
img_rows, img_cols = 28, 28
WIDTH = 64*2
num_layers = 8

input = Input(batch_shape=(batch_size, 1, img_rows, img_cols))
# L0 = Flatten()(input)
# L1 = Dense(WIDTH,activation='tanh')(L0)
# L1 = BatchNormalization(mode=2)(L1)
# m = L1
# #L2 = Dense(WIDTH,activation='sigmoid')(L1)
# for i in range(num_layers):
#     m = BatchNormalization(mode=2)(m)
#     m = Dense(WIDTH/2,activation='tanh')(m)
    
# m = BatchNormalization(mode=2)(m)
# m = Dense(WIDTH,activation='tanh')(m)
# #L3 = merge([model,L1],mode='concat')
# L3 = Dense(WIDTH,activation='linear')(m)
# L3 = BatchNormalization(mode=2)(L3)
# final = Dense(nb_classes,activation='softmax')(L3)
# model = Model(input=input,output=[final], name='m2_snn')
# # firstlayer = Model(input=input,output=L1)
# # lastlayer = Model(input=input,output=L3)

m = Flatten()(input)
m = Dense(WIDTH, activation='tanh')(m)
# m = Dropout(0.2)(m)
m = Dense(WIDTH, activation='tanh')(m)
m = Dense(nb_classes, activation='softmax')(m)

model = Model(input=input, output=[m])

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [39]:
# modelone.__dict__

In [40]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (8, 1, 28, 28)        0                                            
____________________________________________________________________________________________________
flatten_3 (Flatten)              (8, 784)              0           input_3[0][0]                    
____________________________________________________________________________________________________
dense_7 (Dense)                  (8, 128)              100480      flatten_3[0][0]                  
____________________________________________________________________________________________________
dense_8 (Dense)                  (8, 128)              16512       dense_7[0][0]                    
___________________________________________________________________________________________

**Have not tried to run anything below this point!**

In [41]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="/Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/output/mnist_box_shape_snn" + str(loop_num) + ".hdf5", verbose=1, save_best_only=True)
# earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
model.fit(X2,Y2,batch_size=batch_size,nb_epoch=nb_epoch,callbacks=[checkpointer],validation_split=0.2,shuffle=True, verbose=2)
sleep(0.1)

Train on 144 samples, validate on 36 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 2.48054, saving model to /Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/output/mnist_box_shape_snn0.hdf5
0s - loss: 2.6995 - acc: 0.1806 - val_loss: 2.4805 - val_acc: 0.1944
Epoch 2/100
Epoch 00001: val_loss improved from 2.48054 to 2.09753, saving model to /Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/output/mnist_box_shape_snn0.hdf5
0s - loss: 2.2017 - acc: 0.4792 - val_loss: 2.0975 - val_acc: 0.4722
Epoch 3/100
Epoch 00002: val_loss improved from 2.09753 to 1.79077, saving model to /Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-research-notes/output/mnist_box_shape_snn0.hdf5
0s - loss: 1.8480 - acc: 0.6806 - val_loss: 1.7908 - val_acc: 0.5833
Epoch 4/100
Epoch 00003: val_loss improved from 1.79077 to 1.56357, saving model to /Users/fredhohman/Desktop/pnnl/ubuntu-vm/Desktop/Github/pnnl-summer-

In [42]:
# json_string = extractor.to_json()
# outfile = open("output/mnist_box_shape_classifier_only_no_weights_ex.json",'w')
# outfile.write(json_string)
# outfile.close()
# extractor.save_weights('output/mnist_box_shape_classifier_only_no_weights_ex.h5', overwrite=True)

In [43]:
json_string = model.to_json()
outfile = open("output/mnist_box_shape_snn.json",'w')
outfile.write(json_string)
outfile.close()
model.save_weights('output/mnist_box_shape_snn.h5', overwrite=True)

In [44]:
index = 0
blankp = np.zeros([1,28,28])
# print(np.around(model.predict(boxify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(lineify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(circleify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(triangulify_center(np.copy(blankp))[None,])))
# print(np.around(model.predict(noiseify()[None,])))


temp = np.array([boxify_top_left(np.copy(blankp))])
model.predict(temp)

array([[  9.77770448e-01,   5.44763869e-04,   4.16595582e-03,
          1.51734590e-03,   2.55926629e-03,   1.99162238e-03,
          6.96366094e-03,   1.05177816e-04,   8.55255203e-05,
          3.72382608e-04,   1.07280706e-04,   1.99962800e-04,
          2.53162434e-04,   6.61711441e-04,   3.62900348e-04,
          2.16555753e-04,   3.80765414e-04,   1.74152886e-03]], dtype=float32)